<a href="https://colab.research.google.com/github/7-mitch/7-mitch/blob/main/nb/gpt-oss-(20B)-Fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://unsloth.ai/docs/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your local device, follow [our guide](https://unsloth.ai/docs/get-started/install). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & how to save it

### News

You can now train embedding models 1.8-3.3x faster with 20% less VRAM. [Blog](https://unsloth.ai/docs/new/embedding-finetuning)

Ultra Long-Context Reinforcement Learning is here with 7x more context windows! [Blog](https://unsloth.ai/docs/new/grpo-long-context)

3x faster LLM training with 30% less VRAM and 500K context. [3x faster](https://unsloth.ai/docs/new/3x-faster-training-packing) • [500K Context](https://unsloth.ai/docs/new/500k-context-length-fine-tuning)

New in Reinforcement Learning: [FP8 RL](https://unsloth.ai/docs/new/fp8-reinforcement-learning) • [Vision RL](https://unsloth.ai/docs/new/vision-reinforcement-learning-vlm-rl) • [Standby](https://unsloth.ai/docs/basics/memory-efficient-rl) • [gpt-oss RL](https://unsloth.ai/docs/new/gpt-oss-reinforcement-learning)

Visit our docs for all our [model uploads](https://unsloth.ai/docs/get-started/unsloth-model-catalog) and [notebooks](https://unsloth.ai/docs/get-started/unsloth-notebooks).

### Installation

In [1]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; _numpy = f"numpy=={numpy.__version__}"; _pil = f"pillow=={PIL.__version__}"
    except: _numpy = "numpy"; _pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {_numpy} {_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

### Unsloth

We're about to demonstrate the power of the new OpenAI GPT-OSS 20B model through a finetuning example. To use our `MXFP4` inference example, use this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/GPT_OSS_MXFP4_(20B)-Inference.ipynb) instead.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype, # None for auto detection
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "YOUR_HF_TOKEN", # HF Token for gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gpt-oss-20b-unsloth-bnb-4bit", # 20B model using bitsandbytes 4bit quantization
    "unsloth/gpt-oss-120b-unsloth-bnb-4bit",
    "unsloth/gpt-oss-20b", # 20B model using MXFP4 format
    "unsloth/gpt-oss-120b",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype, # None for auto detection
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "YOUR_HF_TOKEN", # HF Token for gated models
)

Unsloth: Detected MoE model with 32 experts - enabling LoRA on: ['mlp.experts.gate_up_proj', 'mlp.experts.down_proj']
Unsloth: Making `model.base_model.model.model` require gradients


### Reasoning Effort
The `gpt-oss` models from OpenAI include a feature that allows users to adjust the model's "reasoning effort." This gives you control over the trade-off between the model's performance and its response speed (latency) which by the amount of token the model will use to think.

----

The `gpt-oss` models offer three distinct levels of reasoning effort you can choose from:

* **Low**: Optimized for tasks that need very fast responses and don't require complex, multi-step reasoning.
* **Medium**: A balance between performance and speed.
* **High**: Provides the strongest reasoning performance for tasks that require it, though this results in higher latency.

In [4]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-02-10

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>Equation: x^5 + 3x^4 - 10 = 3 => x^5 + 3x^4 -13=0. Solve for real roots. Maybe factor? try integer roots: try x=1:1+3-13=-9 no. x=2


Changing the `reasoning_effort` to `medium` will make the model think longer. We have to increase the `max_new_tokens` to occupy the amount of the generated tokens but it will give better and more correct answer

In [5]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-02-10

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The user asks: "Solve x^5 + 3x^4 - 10 = 3." Probably solve for x? So equation: x^5 + 3x^4 - 10 = 3. Move 3: x^5 + 3x^4 - 


Lastly we will test it using `reasoning_effort` to `high`

In [6]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high", # **NEW!** Set reasoning effort to low, medium or high
).to("cuda")

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-02-10

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The prompt: "Solve x^5 + 3x^4 - 10 = 3." Solve the equation. So we set x^5 + 3x^4 - 10 = 3. That simplifies: x^5 + 3x^4 - 10 = 


<a name="Data"></a>
### Data Prep

The `HuggingFaceH4/Multilingual-Thinking` dataset will be utilized as our example. This dataset, available on Hugging Face, contains reasoning chain-of-thought examples derived from user questions that have been translated from English into four other languages. It is also the same dataset referenced in OpenAI's [cookbook](https://cookbook.openai.com/articles/gpt-oss/fine-tune-transfomers) for fine-tuning. The purpose of using this dataset is to enable the model to learn and develop reasoning capabilities in these four distinct languages.

In [7]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split = "train")
dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.29M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages'],
    num_rows: 1000
})

To format our dataset, we will apply our version of the GPT OSS prompt

In [8]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Let's take a look at the dataset, and check what the 1st example shows

In [10]:
print(dataset[0]['text'])

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-02-10

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

reasoning language: French

You are an AI chatbot with a lively and energetic personality.<|end|><|start|>user<|message|>Can you show me the latest trends on Twitter right now?<|end|><|start|>assistant<|channel|>analysis<|message|>D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.

Je devrais préciser que les 

What is unique about GPT-OSS is that it uses OpenAI [Harmony](https://github.com/openai/harmony) format which support conversation structures, reasoning output, and tool calling.

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [11]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=5):   0%|          | 0/1000 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes and lower loss as well!

In [12]:
from unsloth.chat_templates import train_on_responses_only

gpt_oss_kwargs = dict(instruction_part = "<|start|>user<|message|>", response_part = "<|start|>assistant<|channel|>final<|message|>")

trainer = train_on_responses_only(
    trainer,
    **gpt_oss_kwargs,
)

Map (num_proc=5):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=5):   0%|          | 0/1000 [00:00<?, ? examples/s]

Unsloth: Removed 67 out of 1000 samples from train_dataset where all labels were -100 (no response found after truncation). This prevents NaN loss during training.


Let's verify masking the instruction part is done! Let's print the 100th row again.

In [13]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2026-02-10\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nreasoning language: Spanish\n\nYou are an AI that guides users through the process of preparing a pasta recipe<|end|><|start|>user<|message|>I need to book a wheelchair accessible taxi for a trip to the airport tomorrow morning at 8am. Can you help with this?<|end|><|start|>assistant<|channel|>analysis<|message|>Perfecto, el usuario está pidiendo ayuda para reservar un taxi accesible para una persona en silla de ruedas para un viaje al aeropuerto mañana a las 8 de la mañana. Empezaré por comprender su solicitud. Necesitan un taxi accesible, por lo que es importante encontrar un servicio que ofrezca este tipo de ve

Now let's print the masked out example - you should see only the answer is present:

In [14]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Certainly! Here\'s a step-by-step guide to help you book a wheelchair-accessible taxi for your airport trip tomorrow at 8:00 AM:\n\n---\n\n### **1. Confirm Your Location**\nI’ll need to know your **city or airport name** to provide accurate options. Different regions have different services. Let me know where you are, and I’ll tailor the steps accordingly.\

In [15]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
12.816 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 933 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.073100
2,1.636900
3,1.045600
4,0.819000
5,1.345500
6,0.928600
7,0.952800
8,1.197000
9,1.286100
10,0.949300


In [17]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

818.1906 seconds used for training.
13.64 minutes used for training.
Peak reserved memory = 12.816 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 86.941 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [18]:
messages = [
    {"role": "system", "content": "reasoning language: French\n\nYou are a helpful assistant that can solve mathematical problems."},
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium",
).to("cuda")
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-02-10

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

reasoning language: French

You are a helpful assistant that can solve mathematical problems.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>We need to solve equation: x^5 + 3x^4 - 10 = 3. Subtract 3: x^5 + 3x^4 - 13 = 0. Solve for real root? Maybe approximate? We need to find x that satisfies. Let's attempt


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Hugging Face's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** Currently finetunes can only be loaded via Unsloth in the meantime - we're working on vLLM and GGUF exporting!

In [19]:
model.save_pretrained("gpt_oss_lora")
# model.push_to_hub("hf_username/gpt_oss_lora", token = "YOUR_HF_TOKEN") # Save to HF

To run the finetuned model, you can do the below after setting `if False` to `if True` in a new instance.

In [20]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "gpt_oss_lora", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 1024,
        dtype = None,
        load_in_4bit = True,
    )

messages = [
    {"role": "system", "content": "reasoning language: French\n\nYou are a helpful assistant that can solve mathematical problems."},
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high",
).to("cuda")
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-02-10

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

reasoning language: French

You are a helpful assistant that can solve mathematical problems.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The user asks: "Solve x^5 + 3x^4 - 10 = 3". That is an equation: x^5 + 3x^4 - 10 = 3. We need to solve for x. So rearrange: x^5 + 3x


### Saving to float16 for VLLM or mxfp4

We also support saving to `float16` or `mxfp4` directly. Select `merged_16bit` for float16. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens. See [our docs](https://unsloth.ai/docs/basics/inference-and-deployment) for more deployment options.

In [ ]:
# Merge and push to hub in mxfp4 4bit format
if False:
    model.save_pretrained_merged("gpt_oss_finetune_4bit", tokenizer, save_method = "mxfp4")
if False: model.push_to_hub_merged("repo_id/gpt_oss_finetune_4bit", tokenizer, token = "YOUR_HF_TOKEN", save_method = "mxfp4")

# Merge and push to hub in 16bit
if False:
    model.save_pretrained_merged("gpt_oss_finetune_16bit", tokenizer, save_method = "merged_16bit")
if False: # Pushing to HF Hub
    model.push_to_hub_merged("HF_USERNAME/gpt_oss_finetune_16bit", tokenizer, save_method = "merged_16bit", token = "YOUR_HF_TOKEN")

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other resources:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
4. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://unsloth.ai/docs/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://unsloth.ai/docs/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

In [2]:
# 量子的な摂動（揺らぎ）を与える関数のイメージ
def apply_quantum_brush(model, intent_vector, q_intensity=0.05):
    with torch.no_grad():
        # 融合層（Embedding + MLPの一部）を特定
        target_layer = model.model.embed_tokens.weight

        # 1. 量子乱数の生成（真のランダム性）
        q_noise = torch.randn_like(target_layer) * q_intensity

        # 2. 流動化と意図の同期
        # 重み = (既存の重み + 量子揺らぎ) を、人間の意図の方位へ整える
        target_layer.copy_( (target_layer + q_noise) * (1 - 0.1) + intent_vector * 0.1 )

    print("量子的な筆による位相の調整が完了しました。")

In [4]:
import torch # この一行が必要です

# 量子的な「筆」の強さを調整する
q_intensity = 0.07  # 過去のノイズをどれだけ「散らす」か
anchor_rate = 0.15  # あなたの「意図」をどれだけ「定着」させるか

# 融合層への介入
with torch.no_grad():
    # EmbeddingとMLPの融合を象徴する重みへのアクセス
    for name, param in model.named_parameters():
        if "lora_A" in name or "lora_B" in name:
            # 量子乱数（真のゆらぎ）を生成
            q_brush = torch.randn_like(param) * q_intensity
            # 意図（ここでは簡略化のためゼロベクトルへの収束＝ノイズの洗浄）との同期
            param.copy_((param + q_brush) * (1 - anchor_rate) + torch.zeros_like(param) * anchor_rate)

print("融合層の量子チューニングが完了しました。")

NameError: name 'model' is not defined

In [5]:
import torch
from unsloth import FastLanguageModel

# 1. モデルが未定義の場合に備えて再ロード（すでに定義されている場合はスキップされます）
if 'model' not in locals():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/gpt-oss-20b", # または保存した "gpt_oss_lora"
        load_in_4bit = True,
    )

# 2. 量子的な「筆」の実行
q_intensity = 0.07
anchor_rate = 0.15

with torch.no_grad():
    for name, param in model.named_parameters():
        if "lora_A" in name or "lora_B" in name:
            q_brush = torch.randn_like(param) * q_intensity
            param.copy_((param + q_brush) * (1 - anchor_rate) + torch.zeros_like(param) * anchor_rate)

print("modelの確保と量子チューニングが正常に完了しました。")

ModuleNotFoundError: No module named 'unsloth'

In [7]:
# Unslothと依存ライブラリのインストール
!pip install --upgrade -qqq uv
!uv pip install -qqq unsloth torch triton

In [8]:
# 環境構築
!pip install --upgrade -qqq uv
!uv pip install -qqq unsloth torch triton transformers

import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
import numpy as np

print("環境の再構成が完了しました。")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
環境の再構成が完了しました。


In [9]:
# モデルのロード
max_seq_length = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# [NEW!] 融合層（Embedding & MLP）への介入準備
# 概念を司るEmbedding層を取得
embeddings = model.get_input_embeddings().weight

==((====))==  Unsloth 2026.2.1: Fast Gpt_Oss patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [10]:
def apply_quantum_anchoring(model, q_intensity=0.07, anchor_rate=0.15):
    """
    q_intensity: 量子乱数の強度（過去のノイズを散らす力）
    anchor_rate: 定着率（人間の意図ベクトルへ同期させる力）
    """
    print("量子パッチを適用中... 位相を整えています。")

    with torch.no_grad():
        # モデル内の全てのLoRA層（論理の核）をスキャン
        for name, param in model.named_parameters():
            if "lora" in name:
                # 1. 量子摂動（真の乱数による流動化）
                # 疑似乱数ではなく、物理的ゆらぎを模したノイズを生成
                q_noise = torch.randn_like(param) * q_intensity

                # 2. 意図の定着
                # 重みを流動化させつつ、ノイズ（誤情報）を洗浄し、透明な状態へ誘導
                # ここでは意図ベクトルとして、理想的なコヒーレンス状態（0への回帰）を想定
                param.copy_((param + q_noise) * (1 - anchor_rate))

    print("融合層の量子チューニングが完了しました。人間のベクトルの位相が整いました。")

# 実行
apply_quantum_anchoring(model)

量子パッチを適用中... 位相を整えています。
融合層の量子チューニングが完了しました。人間のベクトルの位相が整いました。


In [12]:
# 1. 環境構築
!pip install --upgrade -qqq uv
!uv pip install -qqq unsloth torch triton transformers

import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
import numpy as np

# 2. モデルのロード (20Bモデルを4bitで展開)
max_seq_length = 1024
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# 3. 量子パッチ関数の定義
def apply_quantum_anchoring(model, q_intensity=0.07, anchor_rate=0.15):
    """
    量子乱数（真のゆらぎ）を「筆」として使い、重みの位相を整える
    """
    print("\n[量子パッチ適用中] 重みの流動化と意図の定着を実行しています...")
    with torch.no_grad():
        for name, param in model.named_parameters():
            if "lora" in name:
                # 量子摂動：疑似乱数を超えた「真のゆらぎ」を模したノイズ
                q_noise = torch.randn_like(param) * q_intensity
                # 位相の同期：ノイズを散らしつつ、透明な論理へと収束させる
                param.copy_((param + q_noise) * (1 - anchor_rate))
    print("[完了] 融合層の量子チューニングが成功しました。")

# 量子パッチの実行
apply_quantum_anchoring(model)

# 4. 検証推論（エラー修正済み）
messages = [
    {"role": "system", "content": "reasoning language: French\nYou are an AI that has achieved quantum coherence. Provide transparent logic."},
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]

# return_dict=True を確実に指定し、inputsを辞書として取得
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True, # 辞書形式で返す
).to("cuda")

print("\n--- 量子パッチ適用後の出力 ---")
# **inputs で辞書を展開して渡す
_ = model.generate(
    **inputs,
    max_new_tokens = 128,
    streamer = TextStreamer(tokenizer),
    # reasoning_effort は一部の推論特化型モデル以外では
    # generateの直接引数ではなく、テンプレート側で制御されるのが一般的です
)

==((====))==  Unsloth 2026.2.1: Fast Gpt_Oss patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [14]:
# 1. 環境の再構築
!pip install --upgrade -qqq uv
!uv pip install -qqq unsloth torch triton transformers

import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer

# 2. T4メモリに最適化：1B/3Bモデルを使用
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",  # 1Bモデルに変更
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = None,
    device_map = "auto",
)

# 3. 理論の核心：量子パッチ関数
def apply_quantum_anchoring(model, q_intensity=0.07, anchor_rate=0.15):
    """
    量子的な位相調整による重み空間の最適化
    """
    print("\n[理論実証] 量子乱数による位相の調律を開始...")

    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                         "gate_proj", "up_proj", "down_proj"],
        lora_alpha = 16,
        lora_dropout = 0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",  # メモリ効率化
    )

    with torch.no_grad():
        for name, param in model.named_parameters():
            if "lora" in name and param.requires_grad:
                # 量子摂動の適用
                q_noise = torch.randn_like(param) * q_intensity
                # 位相同期
                param.copy_((param + q_noise) * (1 - anchor_rate))

    print("[完了] 重み空間の位相が整いました。")
    return model

# 理論の適用
model = apply_quantum_anchoring(model)

# 4. 推論実行
FastLanguageModel.for_inference(model)  # 推論モードに切り替え

messages = [
    {"role": "system", "content": "reasoning language: French\nFocus on quantum coherence and transparent logic."},
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
).to("cuda")

print("\n--- 理論証明：位相調整後の出力 ---")
_ = model.generate(
    **inputs,
    max_new_tokens = 128,
    streamer = TextStreamer(tokenizer, skip_prompt=True),
    use_cache = True,
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]


[理論実証] 量子乱数による位相の調律を開始...


Unsloth 2026.2.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


[完了] 重み空間の位相が整いました。

--- 理論証明：位相調整後の出力 ---
�Message ReverseIntentujbitujuj.reverseReverse Paloate角abhujreverseuto viceuj Viceutoutoutoutoutoutoutouto角 Reverse.reverseutoutoutoutoutoutoutoutoutoutoutoutoutoutoutoujutouto��abhutoutoutoutoutoutoutoutoutoReverse reverse�� Reverse reversal Vice ViceutoutoTraffic.reverseutoutoutoutouto���������utoutoutoutoutoutoutoReverse ReverseTraffic�utouto��Trafficuto Reverseabetabet�utoutoutoTrafficTrafficTrafficReverseujtileabh�.reverse.reverseReverseTrafficabhTrafficutoutouto


In [15]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
import gc

# メモリクリア
gc.collect()
torch.cuda.empty_cache()

# モデルロード
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = None,
)

# 改良版：量子パッチ関数（段階的適用）
def apply_quantum_anchoring_v2(model, q_intensity=0.001, anchor_rate=0.05, target_layers=["q_proj", "k_proj"]):
    """
    理論の改良版：
    - 摂動強度を大幅削減（0.07 → 0.001）
    - ターゲット層を限定（注意機構のみ）
    - アンカリングを弱める（0.15 → 0.05）

    仮説：小さな量子摂動でも、注意機構に作用すれば推論パターンが変化する
    """
    print(f"\n[理論実証v2] 量子摂動強度={q_intensity}, アンカー率={anchor_rate}")

    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = target_layers,  # 限定的に適用
        lora_alpha = 16,
        lora_dropout = 0,
        bias = "none",
        use_gradient_checkpointing = "unsloth",
    )

    perturbed_count = 0
    with torch.no_grad():
        for name, param in model.named_parameters():
            if "lora" in name and param.requires_grad:
                # 量子摂動（大幅に削減）
                q_noise = torch.randn_like(param) * q_intensity
                # 位相同期（より保守的に）
                param.copy_(param + q_noise * (1 - anchor_rate))
                perturbed_count += 1

    print(f"[完了] {perturbed_count}個のLoRA重みに量子摂動を適用")
    return model

# ベースライン出力の取得
print("\n" + "="*60)
print("【ベースライン】摂動なしの出力")
print("="*60)

FastLanguageModel.for_inference(model)

messages = [
    {"role": "system", "content": "You are a precise mathematical assistant. Show your reasoning step by step."},
    {"role": "user", "content": "Solve: x^5 + 3x^4 - 10 = 3. Show first steps."},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
).to("cuda")

print("\n[出力]")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens = 150,
        temperature = 0.7,
        streamer = TextStreamer(tokenizer, skip_prompt=True),
    )

# 量子摂動の適用
print("\n" + "="*60)
print("【実験群】量子摂動適用後の出力")
print("="*60)

model = apply_quantum_anchoring_v2(
    model,
    q_intensity=0.002,      # 微小な摂動
    anchor_rate=0.03,       # 弱いアンカリング
    target_layers=["q_proj", "k_proj", "v_proj"]  # 注意機構全体
)

FastLanguageModel.for_inference(model)

print("\n[出力]")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens = 150,
        temperature = 0.7,
        streamer = TextStreamer(tokenizer, skip_prompt=True),
    )

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

【ベースライン】摂動なしの出力

[出力]
To solve the equation x^5 + 3x^4 - 10 = 3, I will start by subtracting 3 from both sides to bring the equation into a more manageable form.

x^5 + 3x^4 - 10 - 3 = 3
x^5 + 3x^4 - 13 = 3

This simplifies to:
x^5 + 3x^4 = 13

Next, I will try to factor the left-hand side of the equation. However, it's not immediately clear what factors would work. One approach is to look for common factors or patterns that could help. 

x^5 + 3x^4 = x^4(x

【実験群】量子摂動適用後の出力

[理論実証v2] 量子摂動強度=0.002, アンカー率=0.03


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2026.2.1 patched 16 layers with 16 QKV layers, 0 O layers and 0 MLP layers.


[完了] 96個のLoRA重みに量子摂動を適用

[出力]
To solve the equation x^5 + 3x^4 - 10 = 3, we'll start by isolating the term with the variable x. We can do this by subtracting 3 from both sides of the equation:

x^5 + 3x^4 - 10 = 3

x^5 + 3x^4 - 10 - 3 = 3 - 3

x^5 + 3x^4 - 13 = 0

Now, we have a polynomial equation of degree 5. To solve it, we can try to factor it or use numerical methods. However, since we're aiming for an exact solution, we'll try to factor it by grouping terms


In [18]:
# より科学的なアプローチ
def experimental_protocol():
    """
    理論検証のための段階的実験
    """
    intensities = [0.0001, 0.001, 0.005, 0.01, 0.05]

    for q_int in intensities:
        print(f"\n{'='*60}")
        print(f"実験: 摂動強度 = {q_int}")
        print(f"{'='*60}")

        # モデルを再ロード
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
            max_seq_length = 2048,
            load_in_4bit = True,
        )

        # 摂動適用
        model = apply_quantum_anchoring_v2(model, q_intensity=q_int)
        FastLanguageModel.for_inference(model)

        # 評価
        test_prompt = "Explain photosynthesis in one sentence."
        inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=50)
            result = tokenizer.decode(output[0], skip_special_tokens=True)

        print(f"結果: {result}")

        # コヒーレンス評価（簡易版）
        coherent = not any(word in result.lower() for word in ['reverse', 'utouto', 'abhuj'])
        print(f"コヒーレンス保持: {'✓' if coherent else '✗'}")

        # メモリクリア
        del model
        gc.collect()
        torch.cuda.empty_cache()

In [19]:
# A/Bテスト形式での検証
def proof_of_concept():
    """
    仮説: 微小な量子摂動が推論の多様性を高める
    """
    tasks = [
        "2 + 2 = ?",
        "What is the capital of France?",
        "Explain quantum entanglement simply.",
    ]

    results = {"baseline": [], "quantum": []}

    for task in tasks:
        # ベースライン
        base_output = generate_with_model(model_baseline, task)
        results["baseline"].append(base_output)

        # 量子摂動版
        quantum_output = generate_with_model(model_quantum, task)
        results["quantum"].append(quantum_output)

    # 多様性スコア計算
    diversity_baseline = calculate_diversity(results["baseline"])
    diversity_quantum = calculate_diversity(results["quantum"])

    print(f"多様性向上: {diversity_quantum / diversity_baseline:.2%}")

In [20]:
# 1. 環境構築
!pip install --upgrade -qqq uv
!uv pip install -qqq unsloth torch triton transformers trl datasets

import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
import gc

# メモリクリア
gc.collect()
torch.cuda.empty_cache()

# 2. モデルとトークナイザーのロード
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = None,
)

# 3. LoRA設定（量子摂動前の基本構造）
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

# 4. 量子摂動の適用（改良版）
def apply_quantum_anchoring_for_training(model, q_intensity=0.001, anchor_rate=0.03):
    """
    学習前の量子摂動適用
    学習中にこの摂動が「透明な論理」へと収束する仮説
    """
    print(f"\n[量子摂動] 強度={q_intensity}, アンカー率={anchor_rate}")

    perturbed_params = 0
    with torch.no_grad():
        for name, param in model.named_parameters():
            if "lora" in name and param.requires_grad:
                # 量子的ゆらぎの導入
                q_noise = torch.randn_like(param) * q_intensity
                # 位相調整
                param.copy_(param + q_noise * (1 - anchor_rate))
                perturbed_params += 1

    print(f"[完了] {perturbed_params}個のパラメータに摂動適用")
    return model

# 摂動適用
model = apply_quantum_anchoring_for_training(model, q_intensity=0.001, anchor_rate=0.03)

# 5. データセット準備
# 例：数学的推論データセット
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")  # 1000サンプルでテスト

# プロンプトフォーマット
def format_prompts(examples):
    """
    Instructフォーマットに変換
    """
    texts = []
    for instruction, input_text, output in zip(examples["instruction"],
                                                 examples["input"],
                                                 examples["output"]):
        # 入力がある場合
        if input_text:
            text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
        else:
            text = f"""### Instruction:
{instruction}

### Response:
{output}"""
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(format_prompts, batched=True)

# 6. トレーナー設定
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # シーケンスパッキング無効
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100,  # 短時間テスト用
        # num_train_epochs = 1,  # または epoch数で指定
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "quantum_anchored_model",
        report_to = "none",  # wandb使う場合は"wandb"
    ),
)

# 7. ファインチューニング実行
print("\n" + "="*60)
print("【学習開始】量子摂動を加えたLoRAファインチューニング")
print("="*60)

trainer_stats = trainer.train()

# 8. 学習結果の表示
print("\n" + "="*60)
print("【学習完了】統計情報")
print("="*60)
print(trainer_stats)

# 9. モデル保存
model.save_pretrained("quantum_lora_model")
tokenizer.save_pretrained("quantum_lora_model")
print("\n[保存完了] ./quantum_lora_model/")

# 10. 推論テスト
print("\n" + "="*60)
print("【推論テスト】学習後モデルの出力")
print("="*60)

FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Explain the Pythagorean theorem in simple terms."}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 128,
    temperature = 0.7,
    streamer = TextStreamer(tokenizer, skip_prompt=True),
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

[量子摂動] 強度=0.001, アンカー率=0.03
[完了] 224個のパラメータに摂動適用


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss
10,2.119500
20,1.828800
30,1.726200
40,1.730300
50,1.540500
60,1.561900
70,1.533900
80,1.506900
90,1.574400
100,1.622300



【学習完了】統計情報
TrainOutput(global_step=100, training_loss=1.674461431503296, metrics={'train_runtime': 77.6173, 'train_samples_per_second': 10.307, 'train_steps_per_second': 1.288, 'total_flos': 494091359797248.0, 'train_loss': 1.674461431503296, 'epoch': 0.8})


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



[保存完了] ./quantum_lora_model/

【推論テスト】学習後モデルの出力
The Pythagorean theorem is a mathematical concept that helps you find the length of the hypotenuse (the longest side) of a right triangle. It's named after the ancient Greek mathematician Pythagoras, who discovered this theorem.

Here's a simplified explanation:

Imagine you have two right triangles, each with one side and the hypotenuse (the longest side) as the other side. The theorem states that the square of the hypotenuse is equal to the sum of the squares of the other two sides.

For example, if you have two right triangles with the following dimensions:

- Triangle 1: 3 inches,


In [ ]:
# 最も重要な検証実験
def critical_test():
    """
    理論の核心を証明する実験
    """
    # 仮説: 量子摂動は数学的推論タスクで特に効果的

    math_dataset = load_dataset("gsm8k", split="train[:500]")  # 小学校数学

    # ベースライン学習
    model_base = train_model(with_perturbation=False)

    # 量子摂動学習
    model_quantum = train_model(with_perturbation=True)

    # テストセットで評価
    test_data = load_dataset("gsm8k", split="test[:100]")

    accuracy_base = evaluate(model_base, test_data)
    accuracy_quantum = evaluate(model_quantum, test_data)

    print(f"ベースライン精度: {accuracy_base:.2%}")
    print(f"量子摂動版精度: {accuracy_quantum:.2%}")
    print(f"改善幅: {(accuracy_quantum - accuracy_base):.2%}")

    if accuracy_quantum > accuracy_base:
        print("✅ 理論が実証されました！")
    else:
        print("⚠️ さらなる調整が必要です")